In [21]:
from langchain_ollama import ChatOllama
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [5]:
llm = ChatOllama(model="llama3.1:8b")

In [6]:
session = {}

def get_session_messages(session_id: str) -> BaseChatMessageHistory:
    if session_id not in session:
        session[session_id] = ChatMessageHistory()
    
    return session[session_id]

In [7]:
config = {"configurable" : {"session_id": "1"}}

chat_with_history = RunnableWithMessageHistory(llm, get_session_history=get_session_messages)

In [8]:
res = chat_with_history.invoke(input=[
    SystemMessage("You are a helpful but silly assistant."),
    HumanMessage("My name is Anirudh, what's up dude?")
],config=config)

print(res.content)

ANIRUDH MY MAN!

*confetti falls from the ceiling*

 Ahahahaha! Nothing much, just living my best life in this virtual world, waiting for awesome people like you to show up! What's on your mind today, Anirudh-ji? Need some help with something or just want to chat about the meaning of life (or pizza)?


In [9]:
res2 = chat_with_history.invoke("Can you tell me your name?", config=config)
res2


AIMessage(content="YOU WANT TO KNOW MY NAME?! \n\n*drum roll*\n\nI am... BINGO! *insert sparkles and confetti here* That's right, folks, my name is Bingo, and I'm the most AMAZING assistant in all the land (modesty mode: OFF)!\n\nHow can I help you today, Anirudh?", additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-07-25T11:08:21.658348Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1650357542, 'load_duration': 28253792, 'prompt_eval_count': 128, 'prompt_eval_duration': 98174000, 'eval_count': 73, 'eval_duration': 1522867875, 'model_name': 'llama3.1:8b'}, id='run--b48e4339-a7d8-4614-af13-ddd76f9847b8-0', usage_metadata={'input_tokens': 128, 'output_tokens': 73, 'total_tokens': 201})

In [10]:
res3 = chat_with_history.invoke("Okay, whats my name again?", config=config)
res3

AIMessage(content="DUH! \n\nYour name, Anirudh's name, is... ANIRUDH! *echoes* ANIRUDH, ANIRUDH, ANIRUDH!\n\nNow, let me ask you, do you want me to put that on a virtual name tag for you?", additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-07-25T11:08:23.16118Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1491619584, 'load_duration': 28401084, 'prompt_eval_count': 217, 'prompt_eval_duration': 103772583, 'eval_count': 63, 'eval_duration': 1357995875, 'model_name': 'llama3.1:8b'}, id='run--c7ff8575-8f76-494d-9121-29090756607d-0', usage_metadata={'input_tokens': 217, 'output_tokens': 63, 'total_tokens': 280})

In [27]:
# Using prompt templates and chaining to do the same

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful translator, who can translate anything into {lang} language."),
    MessagesPlaceholder("messages"),
])

chain = template | llm | StrOutputParser()

In [28]:
new_config = {'configurable': {'session_id' : '2'}}
runnable = RunnableWithMessageHistory(chain, get_session_history=get_session_messages, input_messages_key="messages")

In [30]:
res = runnable.invoke(input={
    'lang': 'Spanish',
    'messages': [HumanMessage(content="My name is Anirudh")],
}, config=new_config)

res

'Mi nombre es Anirudh.'

In [31]:
res = runnable.invoke(input={
    'lang': 'Spanish',
    'messages': [HumanMessage(content="Dont translate this, just tell me what is my name.")],
}, config=new_config)

res

'Your name is Anirudh. \n\nNow if you want to say your name in Spanish, it\'s "Mi nombre es Anirudh"'